In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('Data/merged_data.csv')

C:\ProgramData\Anaconda3\envs\MFI_Assignment_11\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
def compute_features(df_raw):
    df = df_raw.copy()
    
    # Computing quantities
    df['Net Assets'] = df['at'] - df['lct']
    
    # Change these 3
    df['Investment'] = np.where(df['scf'].isin([1,2,3]),
                                df['capx'] + df['ivch'] + df['aqc'] + df['fuseo'] - df['sppe'] - df['siv'],
                                np.where(df['scf'] == 7,df['capx'] + df['ivch'] + df['aqc'] - df['sppe'] - df['siv'] - df['ivstch'] - df['ivaco'],np.nan)
                                )
    df['Change in Working Capital'] = np.where(df['scf'] == 1,
                                               df['wcapc'] + df['chech'] + df['dlcch'],
                                               np.where(df['scf'].isin([2,3]),
                                                        - df['wcapc'] + df['chech'] + df['dlcch'],
                                                        np.where(df['scf'] == 7,
                                                                 - df['recch'] - df['invch'] - df['apalch'] - df['txach'] - df['aoloch'] + df['chech'] - df['fiao'] - df['dlcch'],
                                                                 np.nan)))
    
    df['Internal Cash Flow'] = np.where(df['scf'].isin([1,2,3]),
                                        df['ibc'] + df['xidoc'] + df['dpc'] + df['txdc'] + df['esubc'] + df['sppiv'] + df['fopo'] + df['fsrco'],
                                        np.where(df['scf'] == 7,
                                                 df['ibc'] + df['xidoc'] + df['dpc'] + df['txdc'] + df['esubc'] + df['sppiv'] + df['fopo'] + df['exre'],
                                                 np.nan))
    
    
    df['Financing Deficit'] = df['dv'] + df['Investment'] - df['Change in Working Capital'] - df['Internal Cash Flow']
    df['Net Debt Issued'] = df['dltis'] - df['dltr']
    df['Net Equity Issued'] = df['sstk'] - df['prstkc']
    df['Net External Financing'] = df['Net Debt Issued'] + df['Net Equity Issued']
    df['Total Debt'] = df['dlc'] + df['dltt']
    df['Book Capitalization'] = df['Total Debt'] + df['teq']
    
    df['Current maturity of long-term debt/net assets'] = df['dlc'] / df['Net Assets']
    df['Long Term Debt / Total Assets'] = df['dltt'] / df['at']
    df['Book Leverage'] = (df['dltt'] + df['dlc'])/(df['dltt'] + df['dlc'] + df['seq'])
    df['Cash Dividend / Net Assets'] = df['dv'] / df['Net Assets']
    df['Investment / df[Net Assets'] = df['Investment'] / df['Net Assets']
    df['Change in Working capital / df[Net Assets'] = df['Change in Working Capital'] / df['Net Assets']
    df['Internal Cash Flow / Net Assets'] = df['Internal Cash Flow'] / df['Net Assets']
    df['Financing Deficit / Net Assets'] = df['Financing Deficit'] / df['Net Assets']
    df['Gross LT Debt Issued / Net Assets'] = df['dltis'] / df['Net Assets']
    df['Net Debt Issued / Net Assets'] = df['Net Debt Issued'] / df['Net Assets']
    df['Net Equity Issued / Net Assets'] = df['Net Equity Issued'] / df['Net Assets']
    df['Net External Financing / Net Assets'] = df['Net External Financing'] / df['Net Assets']
    df['Total Debt / Book Capitalization'] = df['Total Debt'] / df['Book Capitalization']
    df['Tangibility'] = df['ppent'] / df['at']
    df['Market Value of Assets / df[Book Value of Assets'] = df['mkvalt'] / df['at']
    df['Log Sales'] = df['sale']
    df['Profitability'] = df['ni'] / df['at']
    df['Current Ratio'] = df['act'] - df['lct']
    df['Quick Ratio'] = (df['che'] + df['rect']) / df['lct']
    df['Debt to Equity'] = df['lt'] / df['teq']
    df['Interest Burden'] = 1 - (df['xint'] / df['oiadp'])
    df['Interest Coverage'] = df['oiadp'] / df['xint']
    df['Leverage'] = df['at'] / df['teq']
    df['Return on Sales (Profit Margin)'] =  df['oiadp'] / df['sale']
    df['Working Capital / Total Assets'] = ( df['act'] - df['lct']) / df['at']
    df['Retained Earnings / Total Assets'] =  df['re'] / df['at']
    df['Earning Before Interest & Tax / Total Assets'] =  df['oiadp'] / df['at']
    df['Market Value of Equity / Total Liabilities'] = ( df['prcc_f']* df['csho']) / df['lt']
    df['Sales / Total Assets'] =  df['sale']/ df['at']
    
    return(df)

data_w = compute_features(data)

In [5]:
def select_features(df):    
    
    # Selecting the feature matrix
    # X = df[[]].to_numpy()
    X = data_w[data_w.columns.difference(['BANKRUPTCY_FLAG'])].to_numpy()  # Change this to subset only the relavant computed colums
    y = df['Y'].to_numpy()
    
    # Standardising the input features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Returning X and y values
    return(X_scaled,y)

In [15]:
data_w.rename(columns = {'BANKRUPTCY_FLAG':'Y'}) if 'BANKRUPTCY_FLAG' in list(data_w.columns)
select_features()

KeyError: 'Y'